In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat

raw_data_path = (
    "/Users/quileesimeon/Dropbox (MIT)/neuropal_data/Whole-Brain Calcium Activity"
)
dataset = "Young Adults"
file_name = "Tail_Activity_OH16230.mat"
# "Head_Activity_OH15500.mat",
# "Head_Activity_OH16230.mat",
# "Tail_Activity_OH16230.mat",

raw_data = loadmat(os.path.join(raw_data_path, dataset, file_name))

files = [_.item() for _ in raw_data["files"].squeeze()]
print(files)
print(len(files))
print()

bilat_neurons = [_.item() for _ in raw_data["neurons"].squeeze()]
print(bilat_neurons)
print(len(bilat_neurons))
print()

is_left_neuron = [_.squeeze().tolist() for _ in raw_data["is_L"].squeeze()]
print(is_left_neuron)
print(len(is_left_neuron))
print(len(is_left_neuron[0]))
print()

norm_traces = [_.squeeze().tolist() for _ in raw_data["norm_traces"].squeeze()]
print(len(norm_traces))  # 108 corresponds to neurons
print(len(norm_traces[0]))  # 7 files x 2 (L/R neuron) = 14
print(norm_traces[0][0].shape)  # 1 x time (neural activity for neuron 0L/R)
print()

['0930_tail_01', '1007_tail_02', '1007_tail_03', '1007_tail_06', '1007_tail_07', '1008_tail_02', '1008_tail_06', '1008_tail_07', '1010_tail_01', '1010_tail_04', '1010_tail_07', '1011_tail_01', '1011_tail_02', '1011_tail_03', '1011_tail_05', '1011_tail_06', '1024_tail_05', '1024_tail_06', '1028_tail_04', '1028_tail_05', '1028_tail_06']
21

['ALN', 'AS11', 'DA7', 'DA8', 'DA9', 'DD6', 'DVA', 'DVB', 'DVC', 'LUA', 'PDA', 'PDB', 'PHA', 'PHB', 'PHC', 'PLM', 'PLN', 'PQR', 'PVC', 'PVN', 'PVP', 'PVQ', 'PVR', 'PVT', 'PVW', 'VA11', 'VA12', 'VD11', 'VD12', 'VD13']
30

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,

In [7]:
# Logic to get things like the other datasets

# This is per .mat file
# Load the raw data
raw_data = loadmat(os.path.join(raw_data_path, dataset, file_name))

# Frames per second
fps = raw_data["fps"].item()
# There several files (each is data for one worm) in each .mat file
files = [_.item() for _ in raw_data["files"].squeeze()]
# The list `bilat_neurons` does not disambiguate L/R neurons, so we need to do that
bilat_neurons = [_.item() for _ in raw_data["neurons"].squeeze()]
# List of lists. Outer list same length as `neuron`. Inner lists are boolean masks for L/R neurons organized by file in `files`.
is_left_neuron = [
    _.squeeze().tolist() for _ in raw_data["is_L"].squeeze()
]  # in each inner list, all L (1) neurons appear before all R (0) neurons. non-bilateral neurons are nan
#  Histogram-normalized neuronal traces linearly scaled and offset so that neurons are comparable
norm_traces = [
    _.squeeze().tolist() for _ in raw_data["norm_traces"].squeeze()
]  # list-of-lists like is_left_neuron

# This part is the extract_data
neuron_IDs = []
traces = []
time_vector_seconds = []

for f, file in enumerate(files):
    neurons = []
    activity = []
    tvec = np.empty(0)
    for i, neuron in enumerate(bilat_neurons):
        # Assign neuron names with L/R and get associated traces
        bilat_bools = is_left_neuron[i]  # tells us if neuron is L/R
        bilat_traces = norm_traces[i]
        assert len(bilat_traces) == len(
            bilat_bools
        ), f"Something is wrong with the data. Traces don't match with bilateral mask: {len(bilat_traces)} != {len(bilat_bools)}"
        righty = None
        if len(bilat_bools) // len(files) == 2:
            # get lateral assignment
            lefty = bilat_bools[: len(bilat_bools) // 2][f]
            righty = bilat_bools[len(bilat_bools) // 2 :][f]
            # get traces
            left_traces = bilat_traces[: len(bilat_traces) // 2][f]
            right_traces = bilat_traces[len(bilat_traces) // 2 :][f]
        elif len(bilat_bools) == len(files):
            # get lateral assignment
            lefty = bilat_bools[:][f]
            righty = None
            # get traces
            left_traces = bilat_traces[:][f]
            right_traces = None
        else:
            raise ValueError(
                f"Something is wrong with the data.\nNeuron: {neuron}. File: {file}."
            )
        if np.isnan(lefty):  # non-bilaterally symmetric neuron
            act = bilat_traces[f].squeeze().astype(float)
            neurons.append(f"{neuron}")
            activity.append(act)
        else:
            if lefty == 1:  # left neuron
                act = left_traces.squeeze().astype(float)
                neurons.append(f"{neuron}L")
                activity.append(act)
            if righty != None:  # right neuron
                act = right_traces.squeeze().astype(float)
                tvec = np.arange(act.size) / fps
                neurons.append(f"{neuron}R")
                activity.append(act)

        # Deal with  time vector which should be the same across all neurons
        if act.size > 0 and act.size > tvec.size:
            tvec = np.arange(act.size) / fps

    # Add neurons to list of neuron_IDs
    neuron_IDs.append(neurons)
    # Reshape activity to be a 2D array
    activity = np.stack(
        [np.empty_like(tvec) * np.nan if act.size == 0 else act for act in activity]
    ).T
    print(activity.shape)
    # Add acitvity to list of traces
    traces.append(activity)
    # Add time vector to list of time vectors
    time_vector_seconds.append(tvec)

(928, 42)
(929, 42)
(928, 42)
(929, 42)
(928, 42)
(929, 42)
(928, 42)
(928, 42)
(929, 42)
(929, 42)
(929, 42)
(929, 42)
(929, 42)
(928, 42)
(928, 42)
(928, 42)
(925, 42)
(929, 42)
(899, 42)
(929, 42)
(928, 42)


In [11]:
print(len(neuron_IDs), len(traces), len(time_vector_seconds))
print()
print(type(neuron_IDs[0]), type(traces[0]), type(time_vector_seconds[0]))
print()
print(len(neuron_IDs[0]), len(traces[0]), len(time_vector_seconds[0]))
print()

21 21 21

<class 'list'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>

